In [1]:
import importlib
import zipfile, os
import requests
import urllib
from collections import namedtuple
_urljoin = urllib.parse.urljoin
_urlencode = urllib.parse.urlencode

requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

In [2]:
class ConverterError(Exception):
    pass

_ConverterSpec = namedtuple('_ConverterSpec', ['jname', 'func', 'cname'])


class RestConverter():
    def __init__(self):
        self._converters = {}
        self._custom_converters = {}
        
    def Register(self, kls):
        tbl = {}
        for row in kls._converter_table:
            if len(row) == 2:
                jname, func = row
                cname = jname
            elif len(row) == 3:
                jname, func, cname = row
            
            spec = _ConverterSpec(jname, func, cname)
            tbl[jname] = spec
        self._converters[kls] = tbl
        return kls  # allow function use as decorator
    
    def RegisterWithConverter(self, func):
        def RegisterCustom(kls):
            self._custom_converters[kls] = func
            return kls
        return RegisterCustom
    
    def _custom_deserialize(self, jobj, kls):
        conv = self._custom_converters[kls]
        return conv(jobj)
        
    def Deserialize(self, jobj, kls):
        try:
            tbl = self._converters[kls]
        except KeyError:
            return self._custom_deserialize(jobj, kls)
        
        obj = kls()
        for key, val in jobj.items():
            spec = tbl.get(key)
            if spec:
                if spec.func in self._converters:
                    val = self.Deserialize(val, spec.func)
                elif spec.func:
                    val = spec.func(val)
                else:
                    pass  # use val as-is
                name = spec.cname
            else:
                name = key
                # pass : use val as-is (string)
            setattr(obj, name, val)
            
        for key, spec in tbl.items():
            if key not in jobj:
                setattr(obj, spec.cname, None)
        return obj
    
    def DeserializeList(self, jobj, kls):
        r = []
        for sub_jobj in jobj:
            obj = self.Deserialize(sub_obj, kls)
            r.append(obj)
        return r
            
RestConvert = RestConverter()     

In [3]:
# class TestCaseStepsContainer:
#     def __init__(self, case):
#         self.test_case = case

class TestStep:
    pass
    
def commentstep_converter(comment):
    step = CommentStep()
    step.text = comment['comment']
    step.type = 'comment'
    return step
    
@RestConvert.RegisterWithConverter(commentstep_converter)
class CommentStep(TestStep):
    pass

@RestConvert.Register
class ExpectedResult:
    _converter_table = [
        ('text', str),
        ('fileReferences', None)
    ]
    
    
def testcasestep_converter(jobj):
    step = jobj['step']
    number = step['number']
    text = step['text']
    results = []
    notes = []
    for row in step['stepRows']:
        if row['type'] == 'expectedResult':
            r = RestConvert.Deserialize(row['expectedResult'], ExpectedResult)
            results.append(r)
        elif row['type'] == 'stepNote':
            notes.append(row['stepNote'])
    
    step = TestCaseStep()
    step.type = 'step'
    step.number = number
    step.text = text
    step.expected_results = results
    step.notes = notes
    return step


@RestConvert.RegisterWithConverter(testcasestep_converter)
class TestCaseStep(TestStep):
    pass
    
def detailed_steps_converter(detailed):
    steps = []
    for jobj in detailed:
        type = jobj['type']
        if type == 'comment':
            step = RestConvert.Deserialize(jobj, CommentStep)
        elif type == "step":
            step = RestConvert.Deserialize(jobj, TestCaseStep)
        else:
            raise ValueError(f"Don't know how to handle '{type}' step")
        steps.append(step)
    return steps
        
    
def steps_converter(steps):
    data = steps['stepsData']
    typ = data['type']
    if typ != 'detailed':
        raise ValueError(f"Don't know how to handle '{typ}' step lists")
    return detailed_steps_converter(data['detailed'])
    
        
@RestConvert.Register
class TestCase:
    _converter_table = [
        ('id', int),
        ('number', int),
        ('tag', str),
        ('self', str),
        ('ttstudioURL', str),
        ('httpURL', str),
        ('fields', list),  # todo
        ('steps', steps_converter)
    ]
    
    def __init__(self):
        self.client = None
    
    def set_client(self, client):
        self.client = client
        

class TestCasesClient:
    def __init__(self, project):
        self.project = project
        self.test_cases = []
        
    def get_one(self, itemID, /, **opts):
        url = f"/testCases/{itemID}"
        rsp = self.project.get(url, **opts)
        return self._parse_case(rsp.json())
        
    def _parse_case(self, jobj):
        case = RestConvert.Deserialize(jobj, TestCase)
        case.set_client(self)
        return case
        
    def get(self, itemID=None, /, **opts):
        if itemID is None:
            return self.get_all(**opts)
        else:
            return self.get_one(itemID, **opts)
        
    def get_all(self, /, **opts):
        url = "/testCases"  
        rsp = self.project.get(url, **opts)
        
        cases = []
        for jobj in rsp.json()['testCases']:
            case = self._parse_case(jobj)
            cases.append(case)
        return cases

@RestConvert.Register
class Project:
    
    def __init__(self):
        self.client = None
        self.headers = {}
    
    _converter_table = [
        ('id', int),
        ('name', str),
        ('uuid', str)
    ]
    
    def getAccessToken(self):
        """ Get AccessBearerToken """
        headers = {'Authorization':'ApiKey '+self.client.apikey}
        path = f"/{self.id}/token"
        rsp = self.client.get(path, headers)
        token = rsp.json()['accessToken']
        self.headers['Authorization'] = 'Bearer ' + token
    
    def set_client(self, client):
        self.client = client
        
    def get(self, path, **opts):
        path = f"/{self.id}{path}"
        return self.client.get(path, self.headers, opts)
        
    @property
    def TestCases(self):
        return TestCasesClient(self)

In [4]:
class HelixClient():
    def __init__(self, url, key):
        if not url.startswith("http"):
            url = "https://"+url
        
        if not url.endswith("/helix-alm/api/v0"):
            raise ValueError("URL must end with '/helix-alm/api/v0'")
        
        self._url = url
        self.apikey = key
        self._sess = requests.Session()
        
    def _rawget(self, url, headers):
        r = self._sess.get(url, headers=headers, verify=False)
        r.raise_for_status()
        return r
    
    def _prep(self, path, opts):
        base = self._url + path
        if opts is not None:
            qs = _urlencode(opts)
            url = f"{base}?{qs}"
        else:
            url = base
        return url
    
    def get(self, path, headers, opts=None):
        url  = self._prep(path, opts)
        return self._rawget(url, headers)
    
    @property
    def Projects(self):
        headers = {
            'Authorization': 'ApiKey ' + self.apikey,
            'Accept': 'application/json'
        }
        rsp = self.get('/projects', headers)
        projects = []
        for p in rsp.json()['projects']:
            proj = RestConvert.Deserialize(p, Project)
            proj.set_client(self)
            proj.getAccessToken()
            projects.append(proj)
        return projects

In [7]:
pub_key = "b0f1e58cb68a9d61cd320df4534d03b12c5a538ee8f5975a5a47944512f14f5b" 
priv_key = "5edab3331ae2316090c126895ae0c52ba6a9b64fa3918ac5f442f264b3e17ea1"
api_key = f"{pub_key}:{priv_key}"
url = "https://pbsbiotech.helixalm.cloud:8443/helix-alm/api/v0"
client = HelixClient(url, api_key)

project = client.Projects[0]
# testcases = project.TestCases.get(expand='steps')
# case = testcases[0]
case = project.TestCases.get(1, expand='steps')

In [8]:
# print(case.id)
for step in case.steps:
    if step.type == 'comment':
        print(step.text)
    elif step.type == 'step':
        print(step.number, step.text, repr([r.text for r in step.expected_results]))

Comment #1
1 Perform IM00153 section 14. ['Cal performed', 'Electronics work']
Comment #2


In [9]:
from officelib.wordlib import Word, wdc

w = Word()

d = w.Documents.Add()

In [10]:
rows = []
for step in case.steps:
    if step.type == 'comment':
        data = ("", step.text, "")
    elif step.type == 'step':
        data = (str(step.number), step.text, "\n".join(r.text for r in step.expected_results))
    rows.append(data)

r = d.Range()
r.MoveStart(wdc.wdStory, 1)
t = d.Tables.Add(r, len(rows) + 1, 3, wdc.wdWord9TableBehavior)

def format_added_table(t):
    # based on macro recording
    d.Style = "Table Grid"
    t.ApplyStyleHeadingRows = True
    t.ApplyStyleLastRow = False
    t.ApplyStyleFirstColumn = True
    t.ApplyStyleLastColumn = False
    t.ApplyStyleRowBands = True
    t.ApplyStyleColumnBands = False

def pt_to_in(pt):
    return pt / 72

def in_to_pt(inch):
    return inch * 72
    
format_added_table(t)

t.Columns(1).Width = in_to_pt(0.5)
t.Columns(2).Width = in_to_pt(3.5)
t.Columns(3).Width = in_to_pt(3.5)
t.Rows.Alignment = wdc.wdAlignRowCenter


for i, row in enumerate(t.Rows):
    if i == 0:
        data = ("#", "Step", "Expected Result")
    else:
        data = rows[i - 1]
    for cell, rd in zip(row.Cells, data):
        cell.Range.Text = rd
    
r.MoveStart(wdc.wdStory, 1)
r.Text = "test\n"
r.MoveStart(wdc.wdParagraph, 1)
r.Text = "test2"

In [176]:
t.Columns(1).Width

22.799999237060547

In [180]:
t.ParagraphAlignment

AttributeError: '<win32com.gen_py.Microsoft Word 16.0 Object Library.Table instance at 0x105390552>' object has no attribute 'ParagraphAlignment'

In [181]:
sorted(t._prop_map_get_)

['AllowAutoFit',
 'AllowPageBreaks',
 'Application',
 'ApplyStyleColumnBands',
 'ApplyStyleFirstColumn',
 'ApplyStyleHeadingRows',
 'ApplyStyleLastColumn',
 'ApplyStyleLastRow',
 'ApplyStyleRowBands',
 'AutoFormatType',
 'Borders',
 'BottomPadding',
 'Columns',
 'Creator',
 'Descr',
 'ID',
 'LeftPadding',
 'NestingLevel',
 'Parent',
 'PreferredWidth',
 'PreferredWidthType',
 'Range',
 'RightPadding',
 'Rows',
 'Shading',
 'Spacing',
 'Style',
 'TableDirection',
 'Tables',
 'Title',
 'TopPadding',
 'Uniform']